# pCrunch's Crunch class

The `Crunch` class is a general analysis tool for batches of time-series based data across multiple environmental conditions (i.e., a full wind speed and turbulence seed sweep). The methods are agnostic to the aeroelastic multibody simulation tool (OpenFAST or HAWC2 or Bladed or QBlade or in-house equivalents). The `AeroelasticOutput` class provides the data containers for each individual simulation.  The `AeroelasticOutput` class provides many analysis capabilities and the `Crunch` class extends them into their batch versions.

The `Crunch` class supports keeping all time series data in memory and a lean "streaming" version where outputs are processed and then deleted, retaining only the critical statistics and analysis outputs.

This file lays out some workflows and showcases capabilities of the `Crunch` class.  It is probably best to walk through the examples of the `AeroelasticOutput` class first.

## Creating a new class instance

The `Crunch` class can be initialized from a list of AeroelasticOutput instances or none, in order to setup a "streaming" analysis.  Pleaes see the AeroelasticOutput example for the various means to initialize one of its instances.  pCrunch provides a reader for OpenFAST output files (both binary and ascii) and common Python data structures are also supported.  To extend pCrunch for use with other aeroelastic multibody codes, users could simply use the `openfast_readers.py` file as a template.  Here are some examples:

In [199]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pCrunch import Crunch, read, FatigueParams

thisdir = os.path.realpath('')
datadir = os.path.join(thisdir, '..', 'pCrunch', 'test', 'data')

# OpenFAST output files
filelist1 = glob.glob( os.path.join(datadir, '*.out') )
filelist1.sort()
filelist2 = glob.glob( os.path.join(datadir, 'DLC1p1', '*.outb') )
filelist2.sort()
print(f"Found {len(filelist1)} and {len(filelist2)} files.")

# Read all outputs into a list
outputs1 = [read(m) for m in filelist1[1:]]
outputs2 = [read(m) for m in filelist2]

# Vector magnitudes
mc = {
    "RootMc1": ["RootMxc1", "RootMyc1", "RootMzc1"],
    "RootMc2": ["RootMxc2", "RootMyc2", "RootMzc2"],
    "RootMc3": ["RootMxc3", "RootMyc3", "RootMzc3"],
}

# Channel-specific fatigue properties
fc = {
    "RootMc1": FatigueParams(slope=10.0, ultimate_stress=6e6, load2stress=5e2, S_intercept=5e7),
    "RootMc2": FatigueParams(slope=10.0, ultimate_stress=6e6, load2stress=5e2, S_intercept=5e7),
    "RootMc3": FatigueParams(slope=10.0, ultimate_stress=6e6, load2stress=5e2, S_intercept=5e7),
}

# Channels to focus on for extreme event tabulation
ec = ["RotSpeed", "RotThrust", "RotTorq"]

# Standard use case with all outputs read prior to use of Crunch.
mycruncher = Crunch(outputs1)

# Can also add some batch data operations in the constructor (many more available in Batch Processing below)
mycruncher_mc = Crunch(outputs2, magnitude_channels=mc, trim_data=[2, 8], fatigue_channels=fc, extreme_channels=ec)

# When planning on adding outputs later, you still need create a Crunch object that is initially empty of data
# The `lean` flag says that the outputs should be processed, but not stored in memory
mycruncher_lean = Crunch(outputs=[], lean=True)

# Can still add the batch operations to be done later when outputs are added
mycruncher_lean_mc = Crunch(outputs=[], lean=True, magnitude_channels=mc, trim_data=[2,8], fatigue_channels=fc, extreme_channels=ec)

Found 5 files.


## Crunching the data

### With full memory storage
The Crunch class can batch process the outputs using one or more processors up to the number available workstation cores.  This computes the essential statistics for each output.

In [200]:
# Process all outputs in parallel
mycruncher.process_outputs(cores=1)

# Process all outputs and override any prior input setting (especially in fatigue calculation)
mycruncher_mc.process_outputs(compute_damage=True)

The key outputs that are stacked together for each output are:

- Summary statistics
- Load ranking
- Extreme event table
- Damage equivalent loads (DELs)
- Palmgren-Miner damage

In [201]:
# The summary stats per each file are here:
mycruncher.summary_stats

,Time,WindVxi,WindVyi,WindVzi,WaveElev,Wave1Vxi,Wave1Vyi,Wave1Vzi,Wave1Axi,Wave1Ayi,...,Anch7Ten,Anch7Ang,Fair8Ten,Fair8Ang,Anch8Ten,Anch8Ang,TipSpdRat,RotCp,RotCt,RotCq
,min,min,min,min,min,min,min,min,min,min,...,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated
DLC2.3_1.out,30.0,8.200,0.0,0.0,-0.812,-0.5420,0.0,-0.7480,-0.7230,0.0,...,0.0,0.0,12876.9250,4834.6250,0.0,0.0,270.594942,11.814832,31.005534,1.591906
DLC2.3_2.out,30.0,8.197,0.0,0.0,-1.055,-0.5438,0.0,-0.5059,-0.3195,0.0,...,0.0,0.0,12910.2725,4830.3545,0.0,0.0,272.806077,12.052701,31.765697,1.613663
DLC2.3_3.out,30.0,8.197,0.0,0.0,-1.135,-0.5403,0.0,-0.5156,-0.2482,0.0,...,0.0,0.0,12897.7175,4832.5860,0.0,0.0,272.238298,11.956121,31.413593,1.604839


In [202]:
# These are indexable by channel, stat:
mycruncher_mc.summary_stats["RootMc1"]

,min,max,std,mean,median,abs,integrated
DLC1.1_0_NREL5MW_OC3_spar_0.outb,5132.619379,9506.307765,951.625642,6949.989138,6701.966537,9506.307765,41705.127675
DLC1.1_0_NREL5MW_OC3_spar_1.outb,3392.613147,7213.755999,950.036038,5371.847381,5513.355924,7213.755999,32235.919447
DLC1.1_0_NREL5MW_OC3_spar_2.outb,2521.676176,6885.215618,1145.602347,4676.240085,4691.594859,6885.215618,28063.101840
DLC1.1_0_NREL5MW_OC3_spar_3.outb,1882.823512,5626.120451,967.896174,3817.917013,3737.812388,5626.120451,22914.213623
DLC1.1_0_NREL5MW_OC3_spar_4.outb,313.022877,5680.152907,1409.901558,3689.474357,3755.551372,5680.152907,22138.544337


In [203]:
mycruncher_mc.summary_stats["RootMc1"]['min']

DLC1.1_0_NREL5MW_OC3_spar_0.outb    5132.619379
DLC1.1_0_NREL5MW_OC3_spar_1.outb    3392.613147
DLC1.1_0_NREL5MW_OC3_spar_2.outb    2521.676176
DLC1.1_0_NREL5MW_OC3_spar_3.outb    1882.823512
DLC1.1_0_NREL5MW_OC3_spar_4.outb     313.022877
Name: min, dtype: float64

In [204]:
# Or by file
mycruncher.summary_stats.loc["DLC2.3_2.out"]

Time       min            30.000000
WindVxi    min             8.197000
WindVyi    min             0.000000
WindVzi    min             0.000000
WaveElev   min            -1.055000
                            ...    
Anch8Ang   integrated      0.000000
TipSpdRat  integrated    272.806077
RotCp      integrated     12.052701
RotCt      integrated     31.765697
RotCq      integrated      1.613663
Name: DLC2.3_2.out, Length: 931, dtype: float64

In [205]:
# Load rankings are manipulations of the summary statistics table
# All channels and statistics are available
mycruncher_mc.get_load_rankings(['RootMc1'],['max'])

,file,channel,stat,val
0,DLC1.1_0_NREL5MW_OC3_spar_0.outb,RootMc1,max,9506.307765


In [206]:
# Damage equivalent loads are found here:
mycruncher_mc.dels

,RootMc1,RootMc2,RootMc3
DLC1.1_0_NREL5MW_OC3_spar_0.outb,3638.231953,3447.817794,3118.938260
DLC1.1_0_NREL5MW_OC3_spar_1.outb,3178.596316,3793.594673,3475.762362
DLC1.1_0_NREL5MW_OC3_spar_2.outb,3639.596231,3954.055428,4626.246778
DLC1.1_0_NREL5MW_OC3_spar_3.outb,3113.961108,4680.992009,5365.254675
DLC1.1_0_NREL5MW_OC3_spar_4.outb,4464.653713,4805.394717,5718.781942


In [207]:
# Palmgren-Miner damage can be viewed with (although it is not computed without a `return_damage=True`
mycruncher_mc.damage

,RootMc1,RootMc2,RootMc3
DLC1.1_0_NREL5MW_OC3_spar_0.outb,2.438120e-14,1.424277e-14,5.226597e-15
DLC1.1_0_NREL5MW_OC3_spar_1.outb,6.316915e-15,3.703911e-14,1.544017e-14
DLC1.1_0_NREL5MW_OC3_spar_2.outb,2.447278e-14,5.605041e-14,2.694258e-13
DLC1.1_0_NREL5MW_OC3_spar_3.outb,5.143788e-15,3.030611e-13,1.185918e-12
DLC1.1_0_NREL5MW_OC3_spar_4.outb,1.888118e-13,3.939507e-13,2.244846e-12


In [208]:
# Extreme events table. For each channel, there is a list of the extreme condition for each output case
mycruncher.extremes

{'Time': [{'Time': 90.0,
   'WindVxi': 9.4,
   'WindVyi': 0.0,
   'WindVzi': 0.0,
   'WaveElev': 0.323,
   'Wave1Vxi': 0.118,
   'Wave1Vyi': 0.0,
   'Wave1Vzi': 0.0362,
   'Wave1Axi': 0.133,
   'Wave1Ayi': 0.0,
   'Wave1Azi': -0.00294,
   'GenPwr': 0.0,
   'GenTq': 0.0,
   'HSSBrTq': 0.0,
   'BldPitch1': 90.0,
   'BldPitch2': 90.0,
   'BldPitch3': 90.0,
   'Azimuth': 219.0,
   'RotSpeed': -0.0142,
   'GenSpeed': -1.39,
   'NacYaw': 0.000215,
   'NacYawErr': -0.507,
   'OoPDefl1': 0.029,
   'IPDefl1': 0.591,
   'TwstDefl1': 0.0,
   'OoPDefl2': -0.108,
   'IPDefl2': 0.186,
   'TwstDefl2': 0.0,
   'OoPDefl3': -0.119,
   'IPDefl3': -1.07,
   'TwstDefl3': 0.0,
   'TwrClrnc1': 40.5,
   'TwrClrnc2': 65.3,
   'TwrClrnc3': 62.7,
   'NcIMUTAxs': 1.4,
   'NcIMUTAys': -0.00524,
   'NcIMUTAzs': -0.0227,
   'TTDspFA': -0.487,
   'TTDspSS': 0.00142,
   'TTDspTwst': 0.0,
   'PtfmSurge': 4.77,
   'PtfmSway': -0.0959,
   'PtfmHeave': 0.207,
   'PtfmRoll': -0.0184,
   'PtfmPitch': -3.12,
   'PtfmYaw': 0.

### Crunching in "lean / streaming" mode

If operating in "lean / streaming" mode, the outputs can either be processed one at a time, or even more lean, the summary statistics themselves can be passed to the `cruncher` object to append to the running list.

In [209]:
# Adding AeroelasticOutput objects in lean / streaming mode
for iout in outputs1:
    mycruncher_lean.add_output( iout ) # Each output is processed without retaining the full time series

# Adding statistics incrementally
results_pool = []
for iout in outputs2:
    iresults = mycruncher_lean_mc.process_single( iout ) # This could be the result of parallelized function
    results_pool.append( iresults )

# After parallel processing is complete, assemble all the statistic for batch analysis
for iresults in results_pool:
    fname, stats, extremes, dels, damage =  iresults
    mycruncher_lean_mc.add_output_stats(fname, stats, extremes, dels, damage)

In [210]:
# Results are the same as the full-memory approach above
mycruncher_lean_mc.summary_stats["RootMc1"]['min']

DLC1.1_0_NREL5MW_OC3_spar_1.outb    3392.613147
DLC1.1_0_NREL5MW_OC3_spar_2.outb    2521.676176
DLC1.1_0_NREL5MW_OC3_spar_3.outb    1882.823512
DLC1.1_0_NREL5MW_OC3_spar_4.outb     313.022877
Name: min, dtype: float64

In [211]:
mycruncher_lean_mc.dels

,RootMc1,RootMc2,RootMc3
DLC1.1_0_NREL5MW_OC3_spar_1.outb,3178.596316,3793.594673,3475.762362
DLC1.1_0_NREL5MW_OC3_spar_2.outb,3639.596231,3954.055428,4626.246778
DLC1.1_0_NREL5MW_OC3_spar_3.outb,3113.961108,4680.992009,5365.254675
DLC1.1_0_NREL5MW_OC3_spar_4.outb,4464.653713,4805.394717,5718.781942


## Integrating outputs with a probability weighting (AEP, Damage, etc)

When running design load cases, not all windspeeds, or other environmental condition, occur with equal likelihood.  pCrunch provides a way to assign a probability to each output.  This probability can then weight a summation to compute annual energy production (AEP), or sum all Palmgren-Miner damages together.  Using a subset of the outputs is also a provided capability.

pCrunch provides a couple different ways to set the probabilities, either:
- Inflow wind speed using a Weibull or Rayleigh distribution for the site
- IEC turbine class with different average wind speeds that define a Weibull distribution
- Users can set the probability values directly.

In [212]:
# Set probability based on wind speed channel name, Weibull distribution average of 7.5 m/s (shape factor input optional)
mycruncher.set_probability_wind_distribution('WindVxi', 7.5, kind='weibull', weibull_k=2.0)

# Or Rayleigh distribution using the same distribution average of 7.5 m/s
mycruncher.set_probability_wind_distribution('WindVxi', 7.5, kind='rayleigh')

# If you only want to use some of the outputs, but not all of them
mycruncher.set_probability_wind_distribution('WindVxi', 7.5, kind='weibull', idx=[0,2])

# If you would rather specify the inflow wind speed directly to use in the probability distribution
mycruncher.set_probability_wind_distribution([8,10,12], 7.5, kind='weibull')

# Can also set the probability based on IEC turbine class, again using a channel name of user input of wind speeds
mycruncher.set_probability_turbine_class('WindVxi', 2)
mycruncher.set_probability_turbine_class([8,10,12], 2)

# A savvy user can set the probability values directly (they will be rescaled to sum to one no matter what)
mycruncher.prob = np.array([0.1, 0.5, 0.4])

# A user can also set their own probability vs. wind speed values
mycruncher.set_probability_wind_distribution([8,10,12], 10., kind='user', v_prob=[4,12,24], probability=[0.1,0.4,0.1])
mycruncher.prob




### Computing AEP

Once the probabilities are set, the user can use them to calculate AEP or total fatigue accumulation across the scenarios represented by each output.  For the AEP calculation, the user must specify the channel name.  Additional loss factors or restriction to certain indices are optional inputs.

In [213]:
# Probability weighted and unweighted AEP values are returned in kWh
mycruncher_mc.set_probability_turbine_class('Wind1VelX', 2)
mycruncher_mc.compute_aep('GenPwr')

(149998517829.0486, 151914046913.74493)

In [214]:
# Or with loss factors and restricted by select outputs
mycruncher.compute_aep('GenPwr', loss_factor=0.15, idx=[0,2])

(32743125284.40001, 32633734981.499996)

### Computing lifetime damage and summary DELs

pCrunch computes a summary damage equivalent load (DEL) and Palmgren-Miner lifetime damage based on the outputs in the list.  For the summary DELs, there is an optional input, `idx` that can be used to select the correct outputs.  For the damage calculation, in addition to `lifetime` scaling, there is an allowance for operational runs with `availability`, parked rotor simulations for downtime with `idx_park` scaled with `(1 - availability)`, and expected number of fault events in a lifetime with `idx_fault` and `n_fault`.
All optional inputs are valid for the damage calculation.

The `process_outputs` function shuld be run before this if the output statisctics were not added in streaming mode.

In [215]:
# Damage calculation does not require a channel name, as it uses the previously computed case-specific and channel-specific values.
dels_tot, dams_tot = mycruncher_mc.compute_total_fatigue(lifetime=30.0, availability=0.9)

In [216]:
dels_tot

,RootMc1,RootMc2,RootMc3
Weighted,3544.999024,3771.608011,3717.241161
Unweighted,3607.007864,4136.370924,4460.996803


In [217]:
dams_tot

,RootMc1,RootMc2,RootMc3
Weighted,0.000004,0.000010,0.000039
Unweighted,0.000007,0.000023,0.000106


In [218]:
# Select indices are also available to restrict the summation
dels_tot, dams_tot = mycruncher_mc.compute_total_fatigue(lifetime=30.0, availability=0.9, idx=[0,2])

## Other Batch Procressing

The Crunch class provides batch extensions of nearly all of the operations offered in the AeroelasticOutputs class.  This includes the add channel or drop channel utilities and all statistical functions.  For the statistics, unlike the AeroelasticOutput class, these batch versions are functions, not data properties.  The result is returned as a list, with each index corresponding to the output list.  Many of these statistics also vary by channel, so there are likely to be nested lists.  Also, some are unavailable in "lean / streaming" mode.

In [219]:
# Adding channel
mycruncher.calculate_channel('LSSGagMya + LSSGagMza', 'Test')

# Adding Load Roses
lr = {'TwrBs': ['TwrBsFxt', 'TwrBsFyt']}
mycruncher.add_load_rose(lr, nsec=6)

# Dropping channels by string wildcard
mycruncher.drop_channel('Fair*')
mycruncher.drop_channel('Anch*')
mycruncher.drop_channel('Spn*')
mycruncher.drop_channel('Root*')
mycruncher.drop_channel('Wave*')
mycruncher.drop_channel('Ptfm*')
mycruncher.drop_channel('Tw*')
mycruncher.drop_channel('Yaw*')

Added channel, TwrBs15
Added channel, TwrBs45
Added channel, TwrBs75
Added channel, TwrBs105
Added channel, TwrBs135
Added channel, TwrBs165
Added channel, TwrBs15
Added channel, TwrBs45
Added channel, TwrBs75
Added channel, TwrBs105
Added channel, TwrBs135
Added channel, TwrBs165
Added channel, TwrBs15
Added channel, TwrBs45
Added channel, TwrBs75
Added channel, TwrBs105
Added channel, TwrBs135
Added channel, TwrBs165


In [220]:
# Indices to the minimum value for each channel
mycruncher.idxmins()

array([[   0,  648,    0,    0,  601,  601,    0,    0,    0,    0,  458,
        1093, 1093,  690, 1200,  705,  484,  686,  447,  673, 1017, 1111,
         752,  409,  753,  188,    0,  722,  486,   99,  606,  707,  689,
        1093,  653,  755,  606,  692],
       [   0,  650,    0,    0,  601,  601,    0,    0,    0,    0,  573,
        1095, 1095,  690, 1200,  707,  612,  687,  449,  674, 1017, 1113,
         817, 1151,  985,  186,    0,  722,  488,  460,  606,  708,  689,
        1095,  653,  755,  606,  693],
       [   0,  650,    0,    0,  601,  601,    0,    0,    0,    0,  460,
        1094, 1094,  690, 1200,  705,  612,  686,  449,  674, 1018, 1110,
         817, 1149,  753,  187,    0,  722,  488,  460,  606,  706,  688,
        1094,  653,  755,  606,  692]])

In [221]:
# Indices to the maximum value for each channel
mycruncher.idxmaxs()

array([[1200,  702,    0,    0,  513,  515,    0,  830,  830,  830,  687,
         636,  638,  564,  877,  457,  665,  434,  763,  484,  712,  755,
         603,  498, 1112,  751,    0,  432,  309,  514,  489,  767,  715,
         648,  492,  431,  489,  755],
       [1200,  705,    0,    0,  507,  513,    0,  830,  830,  830,  460,
         638,  638,  563,  883,  459,  666,  435,  765,  485,  711,  983,
         603,  752, 1114,  818,    0,  607,  314,  516,  488,  761,  715,
         649,  491,  607,  489,  754],
       [1200,  705,    0,    0,  507,  511,    0,  830,  830,  830,  105,
         638,  638,  800,  886,  462,  666,  438,  764,  488,  711,  982,
         603,  752, 1111,  817,    0,  607,   75,  516,  488,  790,  715,
         649,  491,  607,  489,  753]])

In [222]:
# Minimum value of each channel
mycruncher.minima()

array([[ 3.000e+01,  8.200e+00,  0.000e+00,  0.000e+00, -8.120e-01,
        -5.420e-01,  0.000e+00, -7.480e-01, -7.230e-01,  0.000e+00,
        -8.320e-01,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  7.560e-02, -4.930e-02, -4.820e+00,
        -2.130e-02, -5.070e-01, -2.090e+00, -9.380e-01,  0.000e+00,
        -3.080e+00, -8.940e-01,  0.000e+00, -3.440e+00, -1.230e+00,
         0.000e+00,  9.610e+00,  9.390e+00,  9.520e+00, -2.250e+00,
        -1.320e-01, -4.790e-01, -8.200e-01, -5.220e-02,  0.000e+00,
         4.770e+00, -1.300e-01, -1.010e+00, -1.140e-01, -5.440e+00,
        -1.290e+00, -3.510e-01, -1.920e-02, -3.520e-01, -1.270e+02,
        -2.120e+02,  1.130e+02, -3.770e+03, -5.170e+03, -8.000e+01,
        -1.460e+02, -2.050e+02, -1.680e+02, -4.040e+03, -6.630e+03,
        -6.750e+01, -1.060e+02, -2.080e+02, -1.140e+02, -3.770e+03,
        -5.690e+03, -6.670e+01, -5.760e+02, -1.110e+03, -5.460e+01,
        -9.230e+02, -1.600e+03, -5.010e+01, -5.9

In [223]:
# Maximum value of each channel
mycruncher.maxima()

array([[ 9.000e+01,  1.270e+01,  0.000e+00,  0.000e+00,  1.110e+00,
         8.750e-01,  0.000e+00,  6.700e-01,  6.800e-01,  0.000e+00,
         5.370e-01,  2.790e+03,  2.730e+01,  0.000e+00,  9.000e+01,
         9.000e+01,  9.000e+01,  3.600e+02,  1.110e+01,  1.090e+03,
         6.380e-03,  1.290e+00,  4.910e+00,  1.230e+00,  0.000e+00,
         4.580e+00,  1.570e+00,  0.000e+00,  4.720e+00,  2.100e+00,
         0.000e+00,  6.550e+01,  6.540e+01,  6.540e+01,  2.190e+00,
         1.760e-01,  5.190e-01,  7.760e-01,  2.720e-02,  0.000e+00,
         3.720e+01,  4.210e-02,  6.390e-01,  1.930e-01,  5.290e+00,
         5.070e-01,  2.790e-01,  1.030e-02,  3.830e-01,  2.500e+02,
         1.700e+02,  6.750e+02,  4.700e+03,  8.870e+03,  5.820e+01,
         2.320e+02,  1.840e+02,  6.370e+02,  4.510e+03,  8.190e+03,
         5.950e+01,  2.380e+02,  1.560e+02,  6.230e+02,  4.590e+03,
         8.550e+03,  6.670e+01,  1.170e+03,  2.390e+03,  1.250e+01,
         1.510e+03,  2.250e+03,  1.840e+01,  8.0

In [224]:
# Maximum value of absolute values of each channel
mycruncher.absmaxima()

array([[9.000e+01, 1.270e+01, 0.000e+00, 0.000e+00, 2.790e+03, 2.730e+01,
        0.000e+00, 9.000e+01, 9.000e+01, 9.000e+01, 3.600e+02, 1.110e+01,
        1.090e+03, 2.130e-02, 1.290e+00, 4.910e+00, 1.230e+00, 4.580e+00,
        1.570e+00, 4.720e+00, 2.100e+00, 2.250e+00, 1.760e-01, 5.190e-01,
        8.200e-01, 5.220e-02, 0.000e+00, 7.590e+02, 1.140e+03, 1.130e+03,
        2.650e+03, 3.840e+03, 3.590e+03, 8.830e+00, 4.660e-01, 1.130e+00,
        6.240e-02, 4.930e+03],
       [9.000e+01, 1.272e+01, 0.000e+00, 0.000e+00, 2.866e+03, 2.780e+01,
        0.000e+00, 9.000e+01, 9.000e+01, 9.000e+01, 3.595e+02, 1.128e+01,
        1.103e+03, 2.337e-02, 1.418e+00, 4.852e+00, 1.152e+00, 4.701e+00,
        1.658e+00, 4.526e+00, 2.183e+00, 2.668e+00, 1.646e-01, 5.947e-01,
        9.407e-01, 5.507e-02, 0.000e+00, 7.869e+02, 1.128e+03, 1.117e+03,
        2.701e+03, 3.969e+03, 3.930e+03, 8.988e+00, 4.801e-01, 1.171e+00,
        6.365e-02, 5.283e+03],
       [9.000e+01, 1.272e+01, 0.000e+00, 0.000e+00

In [225]:
# The range of data values (max - min)
mycruncher.ranges()

array([[6.000000e+01, 4.500000e+00, 0.000000e+00, 0.000000e+00,
        1.922000e+00, 1.417000e+00, 0.000000e+00, 1.418000e+00,
        1.403000e+00, 0.000000e+00, 1.369000e+00, 2.790000e+03,
        2.730000e+01, 0.000000e+00, 9.000000e+01, 9.000000e+01,
        9.000000e+01, 3.599244e+02, 1.114930e+01, 1.094820e+03,
        2.768000e-02, 1.797000e+00, 7.000000e+00, 2.168000e+00,
        0.000000e+00, 7.660000e+00, 2.464000e+00, 0.000000e+00,
        8.160000e+00, 3.330000e+00, 0.000000e+00, 5.589000e+01,
        5.601000e+01, 5.588000e+01, 4.440000e+00, 3.080000e-01,
        9.980000e-01, 1.596000e+00, 7.940000e-02, 0.000000e+00,
        3.243000e+01, 1.721000e-01, 1.649000e+00, 3.070000e-01,
        1.073000e+01, 1.797000e+00, 6.300000e-01, 2.950000e-02,
        7.350000e-01, 3.770000e+02, 3.820000e+02, 5.620000e+02,
        8.470000e+03, 1.404000e+04, 1.382000e+02, 3.780000e+02,
        3.890000e+02, 8.050000e+02, 8.550000e+03, 1.482000e+04,
        1.270000e+02, 3.440000e+02, 3.64

In [226]:
# Channel indices which vary in time
mycruncher.variable()

array([[ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37],
       [ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37],
       [ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37]])

In [227]:
# Channel indices which are constant in time
mycruncher.constant()

array([[ 2,  3,  6, 26],
       [ 2,  3,  6, 26],
       [ 2,  3,  6, 26]])

In [228]:
# Sum of channel values over time
mycruncher.sums()

array([[ 7.20600000e+04,  1.13490000e+04,  0.00000000e+00,
         0.00000000e+00,  1.45390000e+06,  1.49149000e+04,
         0.00000000e+00,  4.35375000e+04,  4.35375000e+04,
         4.35375000e+04,  2.23657283e+05,  7.74545243e+03,
         7.51099628e+05, -3.46733916e-01,  2.54495832e+02,
         2.32562362e+03,  1.03858640e+01,  2.29799321e+03,
         9.31422620e+01,  2.24562467e+03, -5.28656567e+02,
        -1.09668840e+02, -8.25248500e-01, -8.73048870e+01,
         2.11347150e+02, -1.72728339e+01,  0.00000000e+00,
         4.12023170e+05,  1.50433183e+05,  3.76528104e+05,
         1.34057892e+06,  9.13880480e+05, -3.47473930e+05,
         5.41539388e+03,  2.36486146e+02,  6.20449690e+02,
         3.18652103e+01,  5.66406550e+05],
       [ 7.20600000e+04,  1.13487010e+04,  0.00000000e+00,
         0.00000000e+00,  1.48104500e+06,  1.50961200e+04,
         0.00000000e+00,  4.35375000e+04,  4.35375000e+04,
         4.35375000e+04,  2.25975671e+05,  7.80590576e+03,
         7.57

In [229]:
# Sum of channel values over time to the second power
mycruncher.sums_squared()

array([[4.68450050e+06, 1.07675863e+05, 0.00000000e+00, 0.00000000e+00,
        3.52951220e+09, 3.70704810e+05, 0.00000000e+00, 3.61462425e+06,
        3.61462425e+06, 3.61462425e+06, 5.02760394e+07, 7.52039718e+04,
        7.07203040e+08, 1.54708243e-02, 3.91340106e+02, 9.66142624e+03,
        3.92520251e+02, 9.67394657e+03, 4.35085194e+02, 9.84957290e+03,
        7.97331975e+02, 1.28968734e+03, 3.45014475e+00, 7.72712981e+01,
        2.38852920e+02, 6.74706184e-01, 0.00000000e+00, 2.68450466e+08,
        6.12662865e+08, 7.41562383e+08, 3.59799073e+09, 3.41549721e+09,
        2.65420436e+09, 3.69970113e+04, 1.00909306e+02, 5.92786124e+02,
        1.99075826e+00, 4.15927813e+09],
       [4.68450050e+06, 1.07670244e+05, 0.00000000e+00, 0.00000000e+00,
        3.66867825e+09, 3.80048993e+05, 0.00000000e+00, 3.61462425e+06,
        3.61462425e+06, 3.61462425e+06, 5.12478705e+07, 7.63661217e+04,
        7.18652606e+08, 1.84629219e-02, 4.68085113e+02, 9.93645747e+03,
        4.20467723e+02,

In [230]:
# Sum of channel values over time to the third power
mycruncher.sums_cubed()

array([[ 3.24378090e+08,  1.02653193e+06,  0.00000000e+00,
         0.00000000e+00,  8.60044461e+12,  9.22847278e+06,
         0.00000000e+00,  3.11647398e+08,  3.11647398e+08,
         3.11647398e+08,  1.23752604e+10,  7.54261943e+05,
         6.87823471e+11, -1.55309631e-04,  3.97632701e+02,
         3.65921387e+04,  2.00557331e+01,  3.57721307e+04,
         1.83118610e+02,  3.58096171e+04, -1.90298942e+02,
        -2.55246423e+02, -1.94982634e-02, -9.66024701e+00,
         3.84248475e+01, -2.07043415e-02,  0.00000000e+00,
         1.47701790e+11,  2.56410703e+10,  2.78323280e+11,
         8.42326330e+12,  2.85705259e+12, -1.47447766e+11,
         2.62563212e+05,  4.04494814e+01,  4.89291577e+02,
         1.10493376e-01,  4.08472057e+12],
       [ 3.24378090e+08,  1.02645349e+06,  0.00000000e+00,
         0.00000000e+00,  9.13692517e+12,  9.59022632e+06,
         0.00000000e+00,  3.11647398e+08,  3.11647398e+08,
         3.11647398e+08,  1.26925794e+10,  7.71699289e+05,
         7.04

In [231]:
# Sum of channel values over time to the fourth power
mycruncher.sums_fourth()

array([[2.35556217e+10, 9.84358300e+06, 0.00000000e+00, 0.00000000e+00,
        2.10406009e+16, 2.30121389e+08, 0.00000000e+00, 2.73101502e+10,
        2.73101502e+10, 2.73101502e+10, 3.24906035e+12, 7.64467456e+06,
        6.76030490e+14, 2.75371825e-06, 4.66435637e+02, 1.47734354e+05,
        2.31521846e+02, 1.44888437e+05, 4.42147602e+02, 1.48091112e+05,
        1.16441183e+03, 3.43856709e+03, 2.61039990e-02, 9.79398736e+00,
        8.34199309e+01, 8.28803390e-04, 0.00000000e+00, 9.27353209e+13,
        4.67407255e+14, 6.01989471e+14, 2.05603252e+16, 1.65710113e+16,
        1.20895624e+16, 1.89109701e+06, 1.66195253e+01, 4.55325906e+02,
        6.33486427e-03, 4.18542952e+16],
       [2.35556217e+10, 9.84262551e+06, 0.00000000e+00, 0.00000000e+00,
        2.28840158e+16, 2.42584716e+08, 0.00000000e+00, 2.73101502e+10,
        2.73101502e+10, 2.73101502e+10, 3.34264855e+12, 7.88149761e+06,
        6.98059498e+14, 4.09761614e-06, 6.74266104e+02, 1.55841257e+05,
        2.47886040e+02,

In [232]:
# Second moment of the timeseries for each channel
mycruncher.second_moments()

array([[3.00500000e+02, 3.59754814e-01, 0.00000000e+00, 0.00000000e+00,
        1.47332049e+06, 1.54438492e+02, 0.00000000e+00, 1.69554085e+03,
        1.69554085e+03, 1.69554085e+03, 7.18173603e+03, 2.10260092e+01,
        1.97726013e+05, 1.27982687e-05, 2.80942220e-01, 4.29481654e+00,
        3.26753071e-01, 4.39381075e+00, 3.56254493e-01, 4.70500707e+00,
        4.70131355e-01, 1.06550622e+00, 2.87225454e-03, 5.90547883e-02,
        1.67910823e-01, 3.54943832e-04, 0.00000000e+00, 1.05827656e+05,
        4.94438064e+05, 5.19164233e+05, 1.74988441e+06, 2.26485881e+06,
        2.12628895e+06, 1.04734534e+01, 4.52484287e-02, 2.26690301e-01,
        9.53624575e-04, 3.24076083e+06],
       [3.00500000e+02, 3.59781216e-01, 0.00000000e+00, 0.00000000e+00,
        1.53396197e+06, 1.58448311e+02, 0.00000000e+00, 1.69554085e+03,
        1.69554085e+03, 1.69554085e+03, 7.26822055e+03, 2.13418788e+01,
        2.00862915e+05, 1.52820594e-05, 3.39124941e-01, 4.38884850e+00,
        3.49222036e-01,

In [233]:
# Third moment of the timeseries for each channel
mycruncher.third_moments()

array([[ 0.00000000e+00,  7.24103817e-01,  0.00000000e+00,
         0.00000000e+00,  3.62912268e+07,  1.49199725e+01,
         0.00000000e+00,  2.74556931e+04,  2.74556931e+04,
         2.74556931e+04, -1.66473296e+05, -4.70050944e+01,
        -4.28657625e+07, -1.18208118e-07,  1.42971859e-01,
        -1.74234706e+00,  8.22157997e-03, -2.44120879e+00,
         6.91185242e-02, -3.11276612e+00,  5.47666078e-01,
         8.01220062e-02, -1.03138239e-05,  5.21933133e-03,
        -6.21003135e-02,  1.05002797e-06,  0.00000000e+00,
        -2.63126836e+07, -1.66410311e+08, -2.87365023e+08,
        -2.36967816e+08, -3.23191700e+09,  1.74698413e+09,
        -1.47332873e+01, -6.84088977e-04, -8.18052036e-02,
        -2.58183039e-06, -1.28894512e+09],
       [ 0.00000000e+00,  7.25004318e-01,  0.00000000e+00,
         0.00000000e+00,  5.75053081e+07,  2.43492561e+01,
         0.00000000e+00,  2.74556931e+04,  2.74556931e+04,
         2.74556931e+04, -1.95595710e+05, -4.81499644e+01,
        -4.39

In [234]:
# Fourth moment of the timeseries for each channel
mycruncher.fourth_moments()

array([[1.62540300e+05, 2.36715403e+00, 0.00000000e+00, 0.00000000e+00,
        2.24101427e+12, 2.41720377e+04, 0.00000000e+00, 3.66231300e+06,
        3.66231300e+06, 3.66231300e+06, 1.32215554e+08, 6.00900743e+02,
        5.31410662e+10, 2.14993833e-09, 1.89480734e-01, 2.58201836e+01,
        1.92343218e-01, 2.94031633e+01, 3.33815324e-01, 3.56684146e+01,
        1.34972784e+00, 2.83897495e+00, 2.16987348e-05, 7.77218691e-03,
        8.10136759e-02, 2.67210273e-07, 0.00000000e+00, 2.47388634e+10,
        4.25767462e+11, 5.45777485e+11, 3.54343695e+12, 1.54311265e+13,
        1.10130900e+13, 1.49297135e+02, 2.34715778e-03, 1.13935660e-01,
        1.02522873e-06, 3.29067713e+13],
       [1.62540300e+05, 2.37286981e+00, 0.00000000e+00, 0.00000000e+00,
        2.46147567e+12, 2.55940907e+04, 0.00000000e+00, 3.66231300e+06,
        3.66231300e+06, 3.66231300e+06, 1.33183337e+08, 6.20388043e+02,
        5.49528404e+10, 3.20411689e-09, 2.78088152e-01, 2.65440215e+01,
        2.04329835e-01,

In [235]:
# Mean of channel values over time
mycruncher.means()

array([[ 6.00000000e+01,  9.44962531e+00,  0.00000000e+00,
         0.00000000e+00,  1.27447943e-02,  6.18718068e-03,
         0.00000000e+00,  3.85041907e-03, -1.71907660e-03,
         0.00000000e+00, -2.21445962e-03,  1.21057452e+03,
         1.24187344e+01,  0.00000000e+00,  3.62510408e+01,
         3.62510408e+01,  3.62510408e+01,  1.86225881e+02,
         6.44916939e+00,  6.25395194e+02, -2.88704343e-04,
         2.11903274e-01,  1.93640601e+00,  8.64768027e-03,
         0.00000000e+00,  1.91339985e+00,  7.75539234e-02,
         0.00000000e+00,  1.86979573e+00, -4.40180322e-01,
         0.00000000e+00,  4.72011157e+01,  5.67045379e+01,
         5.72012573e+01, -9.13146045e-02, -6.87134471e-04,
        -7.26934946e-02,  1.75975978e-01, -1.43820432e-02,
         0.00000000e+00,  2.75008493e+01, -4.50295389e-02,
        -1.25443755e-01,  4.38608353e-02,  7.02672090e-01,
        -2.11640713e-01, -2.92449263e-02, -1.70710541e-04,
         3.51910075e-04,  1.07833390e+02,  7.77380516e+0

In [236]:
# Median of channel values over time
mycruncher.medians()

array([[ 6.000e+01,  9.400e+00,  0.000e+00,  0.000e+00,  1.100e-02,
        -3.750e-03,  0.000e+00, -2.100e-02, -2.870e-02,  0.000e+00,
         3.420e-02,  2.270e+03,  2.380e+01,  0.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  2.120e+02,  9.980e+00,  9.680e+02,
         4.710e-05, -7.840e-03,  3.340e+00,  2.970e-03,  0.000e+00,
         3.320e+00,  7.440e-02,  0.000e+00,  3.400e+00, -6.680e-01,
         0.000e+00,  4.270e+01,  6.430e+01,  6.270e+01, -1.150e-01,
         2.810e-03, -9.600e-02,  3.070e-01, -1.460e-02,  0.000e+00,
         2.980e+01, -3.820e-02, -9.350e-02,  5.200e-02,  1.190e+00,
         8.260e-03, -2.740e-02, -9.940e-05, -5.620e-03,  1.570e+02,
         5.200e+01,  2.720e+02, -1.000e+03,  5.850e+03, -3.600e+00,
         1.670e+02,  5.960e+01,  2.970e+02, -1.150e+03,  6.140e+03,
        -2.240e+00,  1.700e+02, -1.300e+02,  2.780e+02,  2.750e+03,
         6.190e+03,  2.710e-01, -3.110e+01,  1.620e+03, -1.230e+00,
        -1.250e+01,  1.610e+03, -1.600e+00, -4.8

In [237]:
# Standard deviation of channel values over time
mycruncher.stddevs()

array([[1.73349358e+01, 5.99795643e-01, 0.00000000e+00, 0.00000000e+00,
        4.10147205e-01, 2.90391370e-01, 0.00000000e+00, 2.88897401e-01,
        2.73251232e-01, 0.00000000e+00, 2.74238405e-01, 1.21380414e+03,
        1.24273284e+01, 0.00000000e+00, 4.11769456e+01, 4.11769456e+01,
        4.11769456e+01, 8.47451239e+01, 4.58541266e+00, 4.44663933e+02,
        3.57746681e-03, 5.30039829e-01, 2.07239392e+00, 5.71623189e-01,
        0.00000000e+00, 2.09614187e+00, 5.96870583e-01, 0.00000000e+00,
        2.16910283e+00, 6.85661254e-01, 0.00000000e+00, 1.63784226e+01,
        1.50980161e+01, 1.38528548e+01, 1.03223361e+00, 5.35934188e-02,
        2.43011910e-01, 4.09769231e-01, 1.88399531e-02, 0.00000000e+00,
        9.29342242e+00, 5.75258137e-02, 3.61501601e-01, 7.66947145e-02,
        2.62966525e+00, 5.30540609e-01, 1.55485716e-01, 3.91719967e-03,
        1.47229017e-01, 9.81649302e+01, 1.17885711e+02, 1.72499127e+02,
        2.41576804e+03, 3.68688426e+03, 2.71874030e+01, 1.038115

In [238]:
# Skew of channel values over time
mycruncher.skews()

/Users/gbarter/devel/pCrunch/pCrunch/aeroelastic_output.py:441: RuntimeWarning: invalid value encountered in divide
  return self.third_moments / np.sqrt(self.second_moments) ** 3


array([[ 0.        ,  3.35576018,         nan,         nan,  0.02029345,
         0.00777382,         nan,  0.39325117,  0.39325117,  0.39325117,
        -0.27352714, -0.48753933, -0.48754515, -2.58178732,  0.96011897,
        -0.1957572 ,  0.04401754, -0.26505921,  0.32505281, -0.30500438,
         1.69897511,  0.07284803, -0.06700162,  0.3636907 , -0.90255932,
         0.15702233,         nan, -0.76430443, -0.47864374, -0.76820434,
        -0.10237062, -0.94819683,  0.5634502 , -0.43467513, -0.07107344,
        -0.75793423, -0.08767213, -0.22093471],
       [ 0.        ,  3.35956359,         nan,         nan,  0.03026816,
         0.01220827,         nan,  0.39325117,  0.39325117,  0.39325117,
        -0.31565828, -0.48836777, -0.48810988, -2.76735613,  1.00020468,
        -0.19547436,  0.00956194, -0.28825562,  0.5556911 , -0.32758553,
         1.80630409,  0.07454257, -0.08179378,  0.16780384, -0.95309717,
         0.26787709,         nan, -0.91116122, -0.49707348, -0.76182184,
   

In [239]:
# Kurtosis of channel values over time
mycruncher.kurtosis()

/Users/gbarter/devel/pCrunch/pCrunch/aeroelastic_output.py:445: RuntimeWarning: invalid value encountered in divide
  return self.fourth_moments / self.second_moments ** 2


array([[ 1.79999834, 18.28998253,         nan,         nan,  1.03240516,
         1.01344961,         nan,  1.27391058,  1.27391058,  1.27391058,
         2.56344305,  1.35921776,  1.35926033, 13.12573258,  2.40066212,
         1.39981322,  1.80151351,  1.52304018,  2.63018059,  1.61125117,
         6.1067167 ,  2.50063083,  2.63019873,  2.22860449,  2.87343223,
         2.12096687,         nan,  2.20892711,  1.74160103,  2.02491168,
         1.1571935 ,  3.00825995,  2.43592899,  1.36104241,  1.14639763,
         2.21714346,  1.12736851,  3.13322305],
       [ 1.79999834, 18.33145515,         nan,         nan,  1.04608352,
         1.01944659,         nan,  1.27391058,  1.27391058,  1.27391058,
         2.52112105,  1.3620659 ,  1.36204239, 13.71969909,  2.41803746,
         1.3780518 ,  1.67543856,  1.5734548 ,  3.04834881,  1.6324344 ,
         6.55469602,  2.93598618,  2.52471202,  2.2649096 ,  3.10214528,
         2.41374665,         nan,  2.36359369,  1.71573472,  2.05254012,
   

In [240]:
# Integration of channel values over time
mycruncher.integrated()

array([[ 3.60000000e+03,  5.66980000e+02,  0.00000000e+00,
         0.00000000e+00,  7.54599900e-01,  3.62890200e-01,
         0.00000000e+00,  2.26287665e-01, -1.08495550e-01,
         0.00000000e+00, -1.25229800e-01,  7.26372500e+04,
         7.45140000e+02,  0.00000000e+00,  2.17462500e+03,
         2.17462500e+03,  2.17462500e+03,  1.11759491e+04,
         3.87023477e+02,  3.75307911e+04, -1.73843208e-02,
         1.27378241e+01,  1.16182206e+02,  5.23843200e-01,
         0.00000000e+00,  1.14816361e+02,  4.66143810e+00,
         0.00000000e+00,  1.12195958e+02, -2.64073283e+01,
         0.00000000e+00,  2.83181200e+03,  3.40322250e+03,
         3.43176800e+03, -5.51224200e+00, -4.00164250e-02,
        -4.36360435e+00,  1.05701075e+01, -8.63069693e-01,
         0.00000000e+00,  1.65082675e+03, -2.70262631e+00,
        -7.53082250e+00,  2.63102816e+00,  4.22279590e+01,
        -1.27220148e+01, -1.75378782e+00, -1.03019180e-02,
         2.23312000e-02,  6.47076332e+03,  4.68342000e+0

In [241]:
# Special instance of the integration that specifically uses
# the Power channel string to integrate over time and calculate energy
mycruncher.compute_energy('GenPwr')

DLC2.3_1.out    72637.250
DLC2.3_2.out    73994.400
DLC2.3_3.out    73453.425
Name: integrated, dtype: float64

In [242]:
# Total travel across simulation- useful for pitch drives and yaw drivers
mycruncher.total_travel('BldPitch1')

array([90., 90., 90.])